In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Synonym Table')

In [ ]:
# fetch data set
df_cas = fh.get_df(os.path.join(hndl.sandbox_dir,'workdf.parquet'))

In [ ]:
# cascnt = df_cas.groupby('bgCAS',as_index=False)['DisclosureId'].count()
cascnt = df_cas.groupby('bgCAS',as_index=False).size()
cascnt.columns = ['cas_number','record_cnt']
# syncnt = df_cas.groupby('IngredientName', as_index=False)['DisclosureId'].count()
syncnt = df_cas.groupby('IngredientName', as_index=False).size()
syncnt.columns = ['synonym','syn_count']

This interactive table below lists synonyms for all chemicals published in FracFocus. This is not an exhaustive set. The table is sort-able by any column (click a second time for reverse sort) and use the **Search** bar to limit what is shown in the table. 

   
Use the search function to find
- alternative names for a given CAS number.  Some of these may be product names. Some CAS numbers have thousands of synonyms.
- possible CAS numbers for a given synonym

These synonyms are the ones provided by the CAS reference site, [SciFinder](http://scifinder.cas.org), a subscription service, and the synonyms from EPA's CompTox database.   It can also be helpful to use
[PubChem](https://pubchem.ncbi.nlm.nih.gov/), a free, public site.  

An additional source is the Table H-3 in the appendix of EPA document [Hydraulic Fracturing for Oil and Gas: Impacts from the Hydraulic Fracturing Water Cycle on Drinking Water Resources in the United States (Final Report)](https://cfpub.epa.gov/ncea/hfstudy/recordisplay.cfm?deid=332990).  This table provides a list of **generic** names commonly used that are not specific enough to resolve to a single CAS number.  The CAS number given in the table for these names is "estab_non_spec".  These ingredient names appear frequently in FracFocus.



|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*synonym*| is the one of the names given by SciFinder and/or CompTox for the material identified by the CAS number|
|*cas_number*| is the CAS registration number of a chemical in the FracFocus data set. **Click on this link** to view summary of this chemical for all of FracFocus (filtered data - up to last major update).  If a | 
|*source*| is the source of the synonym & cas number data|
|*syn_count*| is the number of records where IngredientName **exactly** matches the given *synonym*| 
|*record_cnt*| is the total number of records in the current Open-FF full data set that match bgCAS (cas_number). Note that the presence of a cas_number with 0 records may imply that the cas_number was in the raw FracFocus data, but was a mistake and is corrected in the bgCAS number.  However, we have also included many materials that are on lists of chemicals of concern but HAVE NOT been seen inn FracFocus. Although the CASNumber will be displayed for these materials, no hyper link is available because there is no FracFocus summary to display.| 


## Synonyms for chemicals reported in FracFocus and related

In [ ]:
master_df = fh.get_df(os.path.join(hndl.curr_repo_dir,'curation_files','master_synonym_list.parquet'))

# cmptox_df = grd.get_curated_df(repo_name,'CAS_synonyms_CompTox.parquet')
# syn_df = grd.get_curated_df(repo_name,'CAS_synonyms.parquet')
# syn_df = pd.merge(syn_df,cmptox_df,on=['cas_number','synonym'],how='outer',indicator=True)
# syn_df['source'] = np.where(syn_df._merge=='both','SciFinder & CompTox','?')
# syn_df['source'] = np.where(syn_df._merge=='right_only','SciFinder',syn_df.source)
# syn_df['source'] = np.where(syn_df._merge=='left_only','CompTox',syn_df.source)
# syn_df = syn_df.drop('_merge',axis=1)
syn_df = pd.merge(master_df,syncnt,on='synonym',how='left')
syn_df = pd.merge(syn_df,cascnt,on='cas_number',how='left').reset_index(drop=True)
syn_df.syn_count.fillna(0,inplace=True)
syn_df.record_cnt.fillna(0,inplace=True)
syn_df = syn_df[~syn_df.duplicated()].reset_index(drop=True)
syn_df.cas_number = np.where(syn_df.record_cnt>0,
                             '<b>'+syn_df.cas_number.map(lambda x: th.getCatLink(x,x))+'</b>',
                             syn_df.cas_number)
iShow(syn_df.sort_values('record_cnt',ascending=False),
      maxBytes=0, classes="display compact cell-border",index=False)